# BLE Remote Control

The remote controller is used to drive the robot, but also for configuration. It has:

* an analog joystick (read by the ESP32 ADC)
* 3 quadrature encoded potentiometer that also server as push buttons
* 3 LEDs

The ESP32 reports changes over a BLE UART service and also listens for updates, e.g. of the LEDs, using a simple binary protocol. One button together with an LED acts as "power switch": when off, the ESP32 is put into deepsleep to preserve power when operated from a battery. Pressing the button in this state resumes operation.

## Configure

In [1]:
%connect robot-esp32
%rsync
%rlist

Connected to robot-esp32 @ serial:///dev/ttyUSB0
UPDATE  /lib/ble_advertising.py
UPDATE  /lib/remote/ble_uart.py
     67  Jan 20 10:48 2022  boot.py
                            lib/
   2709  Jan 20 11:17 2022      ble_advertising.py
   1147  Jan 20 10:48 2022      filter.py
   4941  Jan 20 10:48 2022      loop.py
                                remote/
    145  Jan 20 10:48 2022          __init__.py
   2525  Jan 20 11:17 2022          ble_uart.py
    512  Jan 20 10:48 2022          button.py
    922  Jan 20 10:48 2022          encoder.py
    777  Jan 20 10:48 2022          joy_axis.py
    708  Jan 20 10:48 2022          led.py
    137  Jan 20 10:48 2022  main.py
    317  Jan 20 10:48 2022  secrets.py


## Functionality

`loop.py`, called on every boot from `main.py` continuously scans the peripherals (using a few helper classes) and BLE messages.

In [1]:
!cat $IOT_PROJECTS/robot/code/esp32/lib/loop.py

from remote import BLE_UART, JoyAxis, Button, Encoder, LED
from machine import ADC, Pin, deepsleep, reset_cause, PWRON_RESET
from struct import pack, unpack
from time import ticks_ms, ticks_diff, sleep_ms
import esp32

"""
Messages:
--------

Format:
pack('>Bf', ord(code), value)

Codes:

a) MCU -> Pi
   x, y: joystick position
   q, b, c: button pressed counts
         Note: button 1 assigned to 'q' (quit = deepsleep)
   1, 2, 2: encoder counts
   v: v_bat [V]
   h: heartbeat count

b) Pi -> MCU
   R, G, B: led intensities (0 ... 1)
   Q: power down remote (deepsleep)
"""

HARTBEAT_MS =   500     # frequency with which hartbeats are sent [ms]
SHUTDOWN_MS = 60000     # shut down if no connection in specified time [ms]
VBATT_MS    = 60000     # rate at which battery level is sent [ms]

def loop():

    # deepsleep wakeup
    deepsleep_wakeup_pin = Pin(4, mode=Pin.IN, pull=Pin.PULL_UP)
    esp32.wake_on_ext0(deepsleep_wakeup_pin, esp32.WAKEUP_ALL_LOW)

    # BLE
    def rx_cb(data):
    

## Raspberry PI

In [1]:
%%host

import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

import asyncio
from robot import Remote

def exception_handler(loop, context):
    msg = context.get("exception", context["message"])
    print("***** asyncio:", context)
    print("***** msg:", msg)
    
remote = Remote()

asyncio.get_event_loop().set_exception_handler(exception_handler)
asyncio.run(remote.run(peripheral_name='iot49-robot'))




scanning for iot49-robot
_find_uart_device: xiaoxiang BMS A4:C1:38:A7:84:1C: xiaoxiang BMS
_find_uart_device: 26-83-90-94-31-3B 26:83:90:94:31:3B: 26-83-90-94-31-3B
_find_uart_device: 67-74-D0-AD-19-50 67:74:D0:AD:19:50: 67-74-D0-AD-19-50
_find_uart_device: 60-41-16-7A-85-61 60:41:16:7A:85:61: 60-41-16-7A-85-61
_find_uart_device: 74-9D-39-36-D5-27 74:9D:39:36:D5:27: 74-9D-39-36-D5-27
_find_uart_device: 42-66-17-F6-10-BE 42:66:17:F6:10:BE: 42-66-17-F6-10-BE
_find_uart_device: 60-41-16-7A-85-61 60:41:16:7A:85:61: 60-41-16-7A-85-61
_find_uart_device: mpy-uart 30:AE:A4:28:39:F2: mpy-uart
_find_uart_device: 42-66-17-F6-10-BE 42:66:17:F6:10:BE: 42-66-17-F6-10-BE
_find_uart_device: 63-B9-48-F0-3C-A5 63:B9:48:F0:3C:A5: 63-B9-48-F0-3C-A5
_find_uart_device: 67-74-D0-AD-19-50 67:74:D0:AD:19:50: 67-74-D0-AD-19-50
_find_uart_device: 4F-95-36-73-5F-CE 4F:95:36:73:5F:CE: 4F-95-36-73-5F-CE
_find_uart_device: C2-88-BF-39-58-8D C2:88:BF:39:58:8D: C2-88-BF-39-58-8D
_find_uart_device: BMV F5:78:D2:09:01:D

AttributeError: 'BLE_UART' object has no attribute '_client'

In [ ]:
try:
    asyncio.get_event_loop().set_exception_handler(exception_handler)
    asyncio.run(remote.run(peripheral_name='iot49-robot'))
except Exception as e:
    print("*** main:", type(e), e)
print("DONE!")